This file is to calculate EHSA, based on the concept [here](https://pro.arcgis.com/en/pro-app/latest/tool-reference/space-time-pattern-mining/emerginghotspots.htm)

In [1]:
import os

current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
analyze_path = os.path.join(parent_dir, "utils")

os.chdir(analyze_path)

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from utils import get_grid, read_data, read_taiwan_specific

combined_data = read_data()
taiwan, grid_filter = read_taiwan_specific()

In [2]:
hex_grid_full = pd.read_csv("../ComputedDataV7/Grid/grid_data_區級篩選.csv")
import ast
from shapely import wkt
hex_grid_full['geometry'] = hex_grid_full['geometry'].apply(wkt.loads)
hex_grid_full['accident_indices'] = hex_grid_full['accident_indices'].apply(ast.literal_eval)
hex_grid_full = gpd.GeoDataFrame(hex_grid_full, geometry='geometry', crs='EPSG:3826')

In [3]:
grid_ehsa = hex_grid_full[['geometry', 'COUNTYNAME', 'TOWNNAME']]

combined_data['hour'] = (combined_data['發生時間'] // 10000).astype(int)

gdf_accidents = gpd.GeoDataFrame(
    combined_data, 
    geometry=gpd.points_from_xy(combined_data['經度'], combined_data['緯度']),
    crs="EPSG:4326"
).to_crs("EPSG:3826")

gdf_accidents = gdf_accidents.reset_index().rename(columns={'index': 'original_accident_index'})

if 'grid_id' not in grid_ehsa.columns:
    grid_ehsa = grid_ehsa.reset_index().rename(columns={'index': 'grid_id'})

joined = gpd.sjoin(gdf_accidents, grid_ehsa[['grid_id', 'geometry']], how="left", predicate="within")

joined = joined.dropna(subset=['grid_id'])

st_cube = joined.groupby(['grid_id', 'hour']).agg(
    accident_count=('original_accident_index', 'count'),
    accident_indices=('original_accident_index', lambda x: list(x))
).reset_index()

all_grids = grid_ehsa['grid_id'].unique()
all_hours = range(24)
multi_index = pd.MultiIndex.from_product([all_grids, all_hours], names=['grid_id', 'hour'])
full_st_cube = pd.DataFrame(index=multi_index).reset_index()

grid_ehsa = pd.merge(full_st_cube, st_cube, on=['grid_id', 'hour'], how='left')

grid_ehsa['accident_count'] = grid_ehsa['accident_count'].fillna(0).astype(int)
grid_ehsa['accident_indices'] = grid_ehsa['accident_indices'].apply(lambda d: d if isinstance(d, list) else [])

print(grid_ehsa.head())

   grid_id  hour  accident_count accident_indices
0        0     0               0               []
1        0     1               0               []
2        0     2               0               []
3        0     3               0               []
4        0     4               0               []


In [4]:
import libpysal
from esda.getisord import G_Local

unique_grids = hex_grid_full[['geometry']]

w = libpysal.weights.contiguity.Queen.from_dataframe(unique_grids)
w.transform = 'B'

grid_ehsa['gi_zscore'] = 0.0
grid_ehsa['gi_zscore'] = grid_ehsa['gi_zscore'].astype(np.float32)

for h in range(24):
    print(f"Processing hour {h}...")
    current_hour_mask = grid_ehsa['hour'] == h
    current_hour_data = grid_ehsa[current_hour_mask].sort_values('grid_id')

    y = current_hour_data['accident_count'].values.astype(np.float64)
    
    # permutations=0 代表只計算解析解 Z-score 不跑模擬
    lg = G_Local(y, w, transform='B', star=True, permutations=0)

    grid_ehsa.loc[current_hour_mask, 'gi_zscore'] = lg.Zs.astype(np.float32)

print("Gi* Z-score 計算完成")

/var/folders/w2/_g9w5yys0f171q4qqm469z1h0000gn/T/ipykernel_13127/2106718916.py:6: FutureWarning: `use_index` defaults to False but will default to True in future. Set True/False directly to control this behavior and silence this warning
  w = libpysal.weights.contiguity.Queen.from_dataframe(unique_grids)
/Users/wangqiqian/opt/anaconda3/envs/ST-RTA/lib/python3.9/site-packages/libpysal/weights/weights.py:224: UserWarning: The weights matrix is not fully connected: 
 There are 759 disconnected components.
 There are 35 islands with ids: 14819, 39742, 47224, 55249, 64684, 101938, 110219, 113385, 113386, 147426, 162411, 174776, 191728, 201476, 245333, 255384, 272283, 283803, 289525, 324425, 332090, 346395, 353258, 353344, 357045, 357046, 361494, 369304, 398951, 398952, 416152, 416153, 434639, 439103, 443576.
  warnings.warn(message)


Processing hour 0...


/Users/wangqiqian/opt/anaconda3/envs/ST-RTA/lib/python3.9/site-packages/libpysal/weights/weights.py:224: UserWarning: The weights matrix is not fully connected: 
 There are 759 disconnected components.
  warnings.warn(message)


Processing hour 1...
Processing hour 2...
Processing hour 3...
Processing hour 4...
Processing hour 5...
Processing hour 6...
Processing hour 7...
Processing hour 8...
Processing hour 9...
Processing hour 10...
Processing hour 11...
Processing hour 12...
Processing hour 13...
Processing hour 14...
Processing hour 15...
Processing hour 16...
Processing hour 17...
Processing hour 18...
Processing hour 19...
Processing hour 20...
Processing hour 21...
Processing hour 22...
Processing hour 23...
Gi* Z-score 計算完成


In [5]:
from joblib import Parallel, delayed
import pymannkendall as mk

def parallel_mk(grid_id, values):
    if len(np.unique(values)) < 2:
        return grid_id, 0.0, 1.0 
    
    res = mk.original_test(values)
    return grid_id, res.z, res.p

grid_ehsa = grid_ehsa.sort_values(['grid_id', 'hour'])

grouped_data = [(name, group['gi_zscore'].values) for name, group in grid_ehsa.groupby('grid_id')]

print(f"開始並行計算 {len(grouped_data)} 個網格的趨勢...")

results = Parallel(n_jobs=-1, verbose=5)(
    delayed(parallel_mk)(grid_id, values) for grid_id, values in grouped_data
)

開始並行計算 450911 個網格的趨勢...


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  70 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 1408 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 27648 tasks      | elapsed:    4.4s
[Parallel(n_jobs=-1)]: Done 69120 tasks      | elapsed:    9.8s
[Parallel(n_jobs=-1)]: Done 119808 tasks      | elapsed:   16.4s
[Parallel(n_jobs=-1)]: Done 179712 tasks      | elapsed:   22.9s
[Parallel(n_jobs=-1)]: Done 248832 tasks      | elapsed:   31.2s
[Parallel(n_jobs=-1)]: Done 327168 tasks      | elapsed:   40.6s
[Parallel(n_jobs=-1)]: Done 414720 tasks      | elapsed:   50.4s
[Parallel(n_jobs=-1)]: Done 450911 out of 450911 | elapsed:   54.3s finished


In [6]:
grid_ehsa.to_csv("../ComputedDataV7/Grid/grid_ehsa_區級篩選.csv", index=False)

In [ ]:
trend_df = pd.DataFrame(results, columns=['grid_id', 'mk_zscore', 'mk_pvalue']).set_index('grid_id')

## 尖峰、離峰GI分類
這裡針對尖峰非尖峰進行分組，這是為了視覺化以及後續帶入模型

In [ ]:
peak_hours = [7, 8, 17]
# 計算每個網格在尖峰與離峰的平均集聚強度 (Gi* Z-score)
peak_stats = grid_ehsa[grid_ehsa['hour'].isin(peak_hours)].groupby('grid_id')['gi_zscore'].mean()
off_peak_stats = grid_ehsa[~grid_ehsa['hour'].isin(peak_hours)].groupby('grid_id')['gi_zscore'].mean()

trend_df['peak_z'] = peak_stats
trend_df['off_peak_z'] = off_peak_stats

def label_st_pattern(row):
    is_peak_hot = row['peak_z'] > 1.96
    is_off_peak_hot = row['off_peak_z'] > 1.96

    if not is_off_peak_hot and is_peak_hot:
        return "Emergent"      # 離峰安全，尖峰爆發
    elif is_off_peak_hot and is_peak_hot:
        return "Persistent"    # 全天候熱點
    elif is_off_peak_hot and not is_peak_hot:
        return "Dissipated"    # 離峰危險，尖峰反而消失 (SiN)
    else:
        return "Stable_Safe"   # 持續安全

trend_df['target_label'] = trend_df.apply(label_st_pattern, axis=1)

In [ ]:
trend_grid = trend_df.join(hex_grid_full[['geometry', 'COUNTYNAME', 'TOWNNAME', 'num_accidents', 'accident_indices']], on='grid_id', how='left')
# trend_grid.to_csv("../ComputedDataV7/Grid/trend_grid.csv", index=False)

## Space Time Cluster (STC)
基於TOWN的分群

In [ ]:
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

In [ ]:
trend_grid = trend_grid.reset_index()
grid_ehsa_with_town = pd.merge(grid_ehsa, trend_grid[['grid_id', 'TOWNNAME']], on='grid_id', how='inner')

town_st_cube = grid_ehsa_with_town.groupby(['TOWNNAME', 'hour'])['gi_zscore'].mean().reset_index()

town_matrix = town_st_cube.pivot(index='TOWNNAME', columns='hour', values='gi_zscore').fillna(0)

X_town_ts = TimeSeriesScalerMeanVariance().fit_transform(town_matrix.values)

n_clusters_ts = 3
model_town_ts = TimeSeriesKMeans(n_clusters=n_clusters_ts, 
                                 metric="dtw",
                                 metric_params={"global_constraint": "sakoe_chiba", 
                                                "sakoe_chiba_radius": 3},
                                 max_iter=10, 
                                 random_state=42)

town_matrix['DTW_Cluster'] = model_town_ts.fit_predict(X_town_ts)

plt.figure(figsize=(12, 8))
for cluster_id in range(n_clusters_ts):
    cluster_data = town_matrix[town_matrix['DTW_Cluster'] == cluster_id].drop(columns=['DTW_Cluster'])
    mean_curve = cluster_data.mean(axis=0)
    plt.plot(mean_curve.index, mean_curve.values, label=f'Cluster {cluster_id} (n={len(cluster_data)})', linewidth=3)

plt.title("24-Hour Accident Hotspot by Town Clusters (DTW)", fontsize=16)
plt.xlabel("Hour of Day", fontsize=14)
plt.ylabel("Average Gi* Z-score", fontsize=14)
plt.xticks(range(0, 24))
plt.axvspan(7, 9, color='red', alpha=0.1, label='Morning Peak')
plt.axvspan(17, 19, color='orange', alpha=0.1, label='Evening Peak')
plt.legend()
plt.grid(True, linestyle='--', alpha=0.6)
plt.show()

In [ ]:
inertias_ts = []
K_range_ts = range(2, 11)

for k in K_range_ts:
    print(f"計算 DTW K={k} ...")
    ts_km = TimeSeriesKMeans(n_clusters=k, 
                             metric="dtw",
                             metric_params={"global_constraint": "sakoe_chiba", 
                                            "sakoe_chiba_radius": 3},
                             max_iter=10, 
                             random_state=42)
    ts_km.fit(X_town_ts)
    inertias_ts.append(ts_km.inertia_)

plt.figure(figsize=(8, 5))
plt.plot(K_range_ts, inertias_ts, marker='s', linestyle='-', color='r')
plt.title('Elbow Method for 24-Hour (TimeSeriesKMeans DTW)', fontsize=14)
plt.xlabel('Number of Clusters (K)', fontsize=12)
plt.ylabel('Inertia (DTW Distance)', fontsize=12)
plt.xticks(K_range_ts)
plt.grid(True, linestyle=':', alpha=0.6)
plt.show()

In [ ]:
dtw_cluster_df = town_matrix[['DTW_Cluster']].reset_index()
final_grid = pd.merge(trend_grid, dtw_cluster_df, on='TOWNNAME', how='left')
final_grid

In [ ]:
# add final_grid

In [ ]:
final_grid.to_csv("../ComputedDataV7/Grid/trend_grid.csv", index=False)

In [ ]:
cluster_county_towns = final_grid.groupby(['DTW_Cluster', 'COUNTYNAME'])['TOWNNAME'].unique()

clusters = sorted(final_grid['DTW_Cluster'].dropna().unique())

for cluster_id in clusters:
    print(f"================ DTW Cluster {cluster_id} ================")
    county_data = cluster_county_towns[cluster_id]
    
    for county, towns in county_data.items():
        print(f" {county} (共 {len(towns)} 區):")
        print(f"  {', '.join(towns)}")
    print("\n")

In [ ]:
cluster_summary_df = final_grid.groupby(['DTW_Cluster', 'COUNTYNAME']).agg(
    Town_Count=('TOWNNAME', 'nunique'), 
    Town_List=('TOWNNAME', lambda x: ', '.join(x.unique()))
).reset_index()

display(cluster_summary_df)

# cluster_summary_df.to_csv("Cluster_Town_Summary.csv", index=False, encoding="utf-8-sig")

In [ ]:
import seaborn as sns

plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei']
# 1. 計算每個 Cluster 在各縣市的「行政區數量」
cluster_county_matrix = final_grid.groupby(['COUNTYNAME', 'DTW_Cluster'])['TOWNNAME'].nunique().unstack(fill_value=0)

# 2. 為了讓圖表更好看，我們計算「比例」 (每個縣市的行政區，有百分之幾落在各 Cluster)
# 這樣可以避免大縣市(如新北29區)在視覺上壓過小縣市(如嘉義市2區)
cluster_county_pct = cluster_county_matrix.div(cluster_county_matrix.sum(axis=1), axis=0) * 100

plt.figure(figsize=(10, 8))
sns.heatmap(cluster_county_pct, annot=True, fmt=".0f", cmap="Blues", 
            cbar_kws={'label': 'Percentage of Towns in County (%)'},
            linewidths=0.5)

plt.title("Spatial Distribution of DTW Clusters across Counties (%)", fontsize=16, fontweight='bold')
plt.xlabel("DTW Cluster Types", fontsize=14)
plt.ylabel("County", fontsize=14)

plt.xticks(ticks=[0.5, 1.5, 2.5], 
           labels=['Cluster 0\n(Rural/Suburbs)', 'Cluster 1\n(Urban Cores)', 'Cluster 2\n(Industrial/Hubs)'], 
           fontsize=12)

plt.tight_layout()
plt.show()